## This notebook is a walkthrough of how to use extract_motion_energy.py.
This script is a wrapper of [pymoten](https://gallantlab.org/pymoten/), and saves out an .npz file named "me_features_all.npz".  

The first thing you need to do is make a features.json file that defines the parameters of your movie, the parameters of the gabor pyramid you want to use, the download path and the save path. This .json file is currently stored in extracted_features folder, one level up from /motionenergy.

This script saves out:
filters.npz --> list of dictionaries that describe the parameters of each gabor filter  
me_features_all.npz --> all of the features for x_train and x_test  
pyramid.obj --> the pyramid object itself  
movie_gray.npz --> the gray matrix of the movie  
movie_downsampledfeatures.npz -->  the TR-decimated, down-sampled features  
movie_features.npz --> the features for each frame  

#### Here's an example of what should go in the .json file:  



{"hcpmovies": {  
&emsp;      "hdim": 90,                ---> desired horizontal dimension of downsampled image  
&emsp;      "vdim": 128,               ---> desired vertical dimension of downsampled image  
&emsp;      "fps": 24,                 ---> frames per second  
&emsp;      "sd": [0, 90, 180, 270],  ---> spatial directions of gabors (aka motion direction)  
&emsp;      "sf": [0,4],               ---> spatial frequency range for gabors  
&emsp;      "tf": [0,4],               ---> temporal frequency range for gabors  
&emsp;      "downloadpath": "/home/jovyan/shared/hcp-7T_Movies/movie/unzip/Post_20140821_version/", ---> path 2 movies  
&emsp;      "movies": ["7T_MOVIE1_CC1_v2.mp4", ---> list of movie names  
&emsp; &emsp; &emsp; &emsp; &emsp;  "7T_MOVIE2_HO1_v2.mp4",   
&emsp; &emsp; &emsp; &emsp; &emsp;  "7T_MOVIE3_CC2_v2.mp4",   
&emsp; &emsp; &emsp; &emsp; &emsp;  "7T_MOVIE4_HO2_v2.mp4],
&emsp; &emsp; &emsp; &emsp; &emsp;   
&emsp; "savepath": "/home/jovyan/workingdirectory/" ---> where you want to save features  
&emsp; }  
}  

#### Gabor pyramid notes
The package that generates the gabor pyramid ([pymoten](https://gallantlab.org/pymoten/index.html)) uses hdim, vdim, spatial frequency (sf), temporal frequency (tf) and spatial directions (sd) parameters ([as well as a slew of other parameters](https://gallantlab.org/pymoten/autodoc/moten.pyramids.html#moten.pyramids.MotionEnergyPyramid)) to generate the full set of gabor filters used in the gabor pyramid.  
  
You can think of the gabor pyramid as being a huge set of filters with different spatial locations, sizes, orientations, spatial frequencies, and spatial directions (aka motion direction).  

The spatial direction parameter corresponds to the direction of motion the filter prefers, which can be decomposed into orientation. For example, a filter with spatial direction 0º and a filter with spatial direction 180º have the same orientation (90º).

Pymoten pushes the stimuli through the gabor pyramid, and returns features that indicate the amount of motion energy signal passing through each filter at each frame.

#### Before running anything, you'll want to make a conda environment.  
##### Here are the terminal calls I ran to make my conda environment called 'low-level':  

1. This creates the conda environment called lowlevel with python 3.8. I'm using python 3.8 because of numpy deprecation issues.
```
conda create --name lowlevel python=3.8  
```
2. This installs all python-based packages
```
pip install numpy==1.20.1 matplotlib scikit-image scikit-video pymoten pillow==9.4.0 moviepy joblib
```
3. This installs conda packages
```
conda install ipykernel nb_conda_kernels jupyter  
```
4. This installs ffmpeg using the conda-forge channel
```
conda install -c conda-forge ffmpeg  
```

5. This step is important for making sure my lowlevel conda environment is available in jupyter hub notebook
```
python -m ipykernel install --user --name lowlevel
```

#### If you want to use a python notebook

In [2]:
import utils
import numpy as np

In [3]:
json_filepath = '/home/jovyan/visual-feature-decoding/extract_features/feature.json'

the following function takes A LONG TIME to run!  
the image resizing function is the bear -- roughly 1.5 hours for a 22k frame movie.

In [4]:
utils.movie_to_gray_array(json_filepath)

7T_MOVIE1_CC1_v2 gray movie already exists!
7T_MOVIE2_HO1_v2 gray movie already exists!
7T_MOVIE3_CC2_v2 gray movie already exists!
7T_MOVIE4_HO2_v2 gray movie already exists!


After that's done, we can push the gray frames through the pyramid, and downsample to the TR  
(24 frames --> 1 frame)

In [5]:
utils.push_thru_pyramid(json_filepath)

saved pyramid!
saved filters!
down_sampled features for 7T_MOVIE1_CC1_v2.mp4 already done!
down_sampled features for 7T_MOVIE2_HO1_v2.mp4 already done!
down_sampled features for 7T_MOVIE3_CC2_v2.mp4 already done!
down_sampled features for 7T_MOVIE4_HO2_v2.mp4 already done!


In [6]:
utils.save_cleaned_features(json_filepath)

In [7]:
x_test = np.load("/home/jovyan/workingdirectory/me_features_all.npz", allow_pickle=True)['x_test']
x_train = np.load("/home/jovyan/workingdirectory/me_features_all.npz", allow_pickle=True)['x_train']


shape of x_test and x_train

In [8]:
x_test.shape, x_train.shape

((4, 1580, 83), (1580, 2885))

below shows the parameters for a single filter.  
more info on filter paramters that we set can be found in json file.

In [15]:
np.load("/home/jovyan/workingdirectory/filters.npz", allow_pickle=True)['filters'][0]


{'centerh': 0.7111111111111111,
 'centerv': 0.5,
 'direction': 0.0,
 'spatial_freq': 4.0,
 'spatial_env': 0.15,
 'temporal_freq': 0.0,
 'temporal_env': 0.3,
 'filter_temporal_width': 16.0,
 'aspect_ratio': 1.4222222222222223,
 'stimulus_fps': 24.0,
 'spatial_phase_offset': 0.0}

#### if you want to call from the command line  

1. first activate the conda environment!
```
conda activate lowlevel
```  

2. then run the following code in the terminal, replacing your json_filepath with the correct one :-)
```
python extract_motion_energy.py '/home/jovyan/visual-feature-decoding/extract_features/motionenergy/feature.json'
```